In [1]:
import tensorflow as tf
import numpy as np
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D,AvgPool2D,Bidirectional,LSTM,GlobalAveragePooling1D,TimeDistributed
import pandas as pd
from tensorflow import keras
from sklearn.metrics import precision_recall_fscore_support

In [2]:
X=sorted(glob.glob("/content/drive/MyDrive/MLSP_A2/X/*.npy"))
Y=sorted(glob.glob("/content/drive/MyDrive/MLSP_A2/Y/*.npy"))

In [3]:
X_test=sorted(glob.glob("/content/drive/MyDrive/MLSP_A2/Validation/X/*.npy"))
Y_test=sorted(glob.glob("/content/drive/MyDrive/MLSP_A2/Validation/Y/*.npy"))

In [4]:
def normalise(arr):
  mu=np.mean(arr,axis=0)
  std_dav=np.std(arr,axis=0)
  normalised_arr=(arr-mu)/std_dav
  return normalised_arr

In [5]:
model=Sequential([
    #Conv1D(128,64,activation='relu',input_shape=(1000, 64),padding='same'),
    #Conv1D(64,32,activation='relu',padding='same'),
    #Conv1D(32,16,activation='relu',padding='same'),
    Bidirectional(LSTM(256,input_shape=(1000,64),return_sequences=True)),
    Bidirectional(LSTM(256,return_sequences=True)),
    #GlobalAveragePooling1D(),
    #AvgPool1D()
    #Dense(512,activation='relu'),
    TimeDistributed(Dense(256,activation='relu')),
    TimeDistributed(Dense(64,activation='relu')),
    TimeDistributed(Dense(11,activation='sigmoid'))
])

In [6]:
BCE_loss=tf.keras.losses.BinaryCrossentropy()
optimizer=tf.keras.optimizers.Adam(.0001)

In [7]:
model.compile(
    loss=BCE_loss,
    optimizer=optimizer,
    metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
)

In [8]:
'''
a=np.squeeze(np.load(X[0])).T
a=a.reshape(20,50,64)
print(a.shape)
a=normalise(a)
print(a.shape)
'''

'\na=np.squeeze(np.load(X[0])).T\na=a.reshape(20,50,64)\nprint(a.shape)\na=normalise(a)\nprint(a.shape)\n'

In [9]:
'model(a).shape'

'model(a).shape'

In [10]:
'''
def train_step_l(data_path, label_path, optimizer, batch_size, shuffle = True):
    index_list = np.arange(len(data_path))
    all_y=[]
    all_y_pred=[]
    if shuffle:
        np.random.shuffle(index_list)
    #print(index_list)
    for sample_data in range (int(np.floor(len(data_path)))):
        #print(step)
        x= np.squeeze(np.load(data_path[sample_data])).T
        #print(x.shape)
        x= x.reshape(20,50, 64)
        y=np.load(label_path[sample_data]).T
        #print(y.shape)
        y = y.reshape(20,50,11)
        x=normalise(x)
        #print(batch_x.shape)
        #if step==3:
        #  break
        #batch_x=np.squeeze(np.transpose(batch_x, (0,1,3,2)))
        x=tf.convert_to_tensor(x)
        y=tf.convert_to_tensor(y)
        with tf.GradientTape() as tape:
         y_pred=model(x)
         #  print(y_pred.shape)
         loss=BCE_loss(y_pred,y)
        grads=tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        if (sample_data+1)%1000==0:
          print("Step = {}, Loss = {}".format(sample_data + 1, loss))
        y_pred=y_pred.numpy()
        y_pred=y_pred.reshape(1000,11)
        y_pred=np.round(y_pred)
        #print(y_pred.shape)
        y=y.numpy()
        y=y.reshape(1000,11)
        #print(y.shape)
        all_y.append(y)
        all_y_pred.append(y_pred)
        
    all_y=np.concatenate(all_y, axis=0)
    all_y_pred=np.concatenate(all_y_pred,axis=0)
    #print (all_y.shape)
    #print (all_y_pred.shape)
    classwise_precision=[]
    classwise_recall=[]
    classwise_f1=[]
    for i in range(11):
     Yk=all_y[:,i]
     Ypredk=all_y_pred[:,i]
     scores=precision_recall_fscore_support(Yk, Ypredk, average='macro')
     classwise_precision.append(scores[0])
     classwise_recall.append(scores[1])
     classwise_f1.append(scores[2])
    avg_precision=sum(classwise_precision)/len(classwise_precision)
    avg_recall=sum(classwise_recall)/len(classwise_recall)
    avg_f1=sum(classwise_f1)/len(classwise_f1)      
    #print("classwise precision:{},avg precision:{},classwise recall:{},avg recall:{},classwise_f1:{},avg f1:{}".format(classwise_precision,avg_precision,classwise_recall,avg_recall,classwise_f1,avg_f1))        

    return  loss,classwise_precision,classwise_recall,classwise_f1,avg_precision,avg_recall,avg_f1
'''

In [11]:

def train_step(data_path, label_path, optimizer, epoch, shuffle = True):
    index_list = np.arange(len(data_path))
    
    if shuffle:
        np.random.shuffle(index_list)
    #print(index_list)
    for sample_data in range (int(np.floor(len(data_path)))):
        #print(step)
        x= np.squeeze(np.load(data_path[sample_data])).T
        #print(x.shape)
        x= x.reshape(20,50, 64)
        y=np.load(label_path[sample_data]).T
        #print(y.shape)
        y = y.reshape(20,50,11)
        x=normalise(x)
        #print(batch_x.shape)
        #if step==3:
        #  break
        x=tf.convert_to_tensor(x)
        y=tf.convert_to_tensor(y)
        with tf.GradientTape() as tape:
         output=model(x)
         #  print(output.shape)
         loss=BCE_loss(output,y)
        grads=tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        if (sample_data+1)%1000==0:
          print("Step = {}, Loss = {}".format(sample_data + 1, loss))
          model.save(f'/content/drive/MyDrive/MLSP_A2/FrameWise_Bi_LSTM_Modified_{epoch}_{sample_data}')
       



    return loss

In [12]:
def eval_step(data_path,label_path):
  all_y=[]
  all_y_pred=[]
  for sample in range(len(data_path)):
    x= np.squeeze(np.load(data_path[sample])).T
    x= x.reshape(20,50, 64)
    x=normalise(x)
    y_pred=model(x)
    y_pred=y_pred.numpy()
    y_pred=y_pred.reshape(1000,11)
    y_pred=y_pred.T
    y_pred=np.round(y_pred)
    #print(y_pred.shape)
    y=np.load(label_path[sample])
    #print(y.shape)
    all_y.append(y.T)
    all_y_pred.append(y_pred.T)
    
  all_y=np.concatenate(all_y, axis=0)
  all_y_pred=np.concatenate(all_y_pred,axis=0)
  #print(all_y.shape,all_y_pred.shape)  
  
  classwise_precision=[]
  classwise_recall=[]
  classwise_f1=[]
  for i in range(11):
    Yk=all_y[:,i]
    Ypredk=all_y_pred[:,i]
    scores=precision_recall_fscore_support(Yk, Ypredk, average='micro')
    classwise_precision.append(scores[0])
    classwise_recall.append(scores[1])
    classwise_f1.append(scores[2])
  avg_precision=sum(classwise_precision)/len(classwise_precision)
  avg_recall=sum(classwise_recall)/len(classwise_recall)
  avg_f1=sum(classwise_f1)/len(classwise_f1)      
  print("classwise precision:{},avg precision:{},classwise recall:{},avg recall:{},classwise_f1:{},avg f1:{}".format(classwise_precision,avg_precision,classwise_recall,avg_recall,classwise_f1,avg_f1))

  return avg_precision, avg_recall, avg_f1




In [ ]:
epochs=20
'''
Loss=[]
Avg_precision=[]
Avg_recall=[]
Avg_f1=[]
Epoch=[]
'''
for epoch in range(epochs):
  loss =train_step(X,Y,optimizer,epoch)
  print('Epoch : {}, Loss : {}'.format(epoch + 1, loss))

  '''
  avg_precision, avg_recall, avg_f1 = eval_step(X_test, Y_test)
  
  Epoch.append(epoch+1)
  Loss.append(loss)
  Avg_precision.append(avg_precision)
  Avg_recall.append(avg_recall)
  Avg_f1.append(avg_f1)
  metrics=pd.DataFrame({
    'Epoch':Epoch,
    'Loss':Loss,
    'Average Precision':Avg_precision,
    'Average Recall':Avg_recall,
    'Average F1':Avg_f1
    })
  metrics.to_csv(f'/content/drive/MyDrive/MLSP_A2/logs/BiLSTM_Metrics.csv')
  '''
  